# DEEZER API CODE
## Import des datas et librairies
En Python2 only, pour pouvoir supporter la librairie unirest !

In [1]:
import unirest
import pandas as pd
import numpy as np
import sys

df = pd.read_csv('./dsg/train.csv')

musics = pd.DataFrame(columns = ['song_id','bpm','rank','gain','lyrics_explicits'])

artists = pd.DataFrame(columns = ['artist_id','nb_fan','nb_album','radio'])

## Requêtes des infos des musiques
ça peut crash de manière random autour de 10 000 et 50 000 calls

In [2]:
def get_track(ids):
    global musics
    compteur = 0
    compteur2 = 0
    for idd in ids:
        response = unirest.get("https://deezerdevs-deezer.p.mashape.com/track/"+str(idd),
        headers={
        "X-Mashape-Key": "eb4DzzTnELmshvFUju71GssNIry0p1tmYUajsnNUoQx2DDVkUN",
        "Accept": "text/plain"
        })
        try:
            musics.loc[len(musics)] = [idd,response.body['bpm'],response.body['rank'],response.body['gain'],response.body['explicit_lyrics']]
            #print([len(musics)-1,idd,response.body['bpm'],response.body['rank'],response.body['gain'],response.body['explicit_lyrics']])
        except KeyError:
            musics.loc[len(musics)] = [idd,np.nan,np.nan,np.nan,np.nan]
            #print([len(musics)-1,idd,np.nan,np.nan,np.nan,np.nan])
        compteur += 1
        if compteur % 50000 == 0:
            compteur2 +=1
            musics.to_csv("./enrich"+str(compteur2*compteur)+".csv")
            compteur = 0
            musics = pd.DataFrame(columns = ['song_id','bpm','rank','gain','lyrics_explicits'])

In [ ]:
get_track(df.media_id.unique())

## Requêtes des infos des artistes

In [3]:
def get_artists(ids):
    global artists
    for idd in ids:
        response = unirest.get("https://deezerdevs-deezer.p.mashape.com/artist/"+str(idd),
        headers={
        "X-Mashape-Key": "eb4DzzTnELmshvFUju71GssNIry0p1tmYUajsnNUoQx2DDVkUN",
        "Accept": "text/plain"
        })
        try:
            artists.loc[len(artists)] = [idd,response.body['nb_fan'],response.body['nb_album'],response.body['radio']]
        except KeyError:
            artists.loc[len(artists)] = [idd,np.nan,np.nan,np.nan]

In [ ]:
get_artists(df.artist_id.unique())

In [ ]:
artists.to_csv('./dsg/musics.csv',index=False)